In [2]:
import matplotlib.pyplot as plt
import cv2
import open3d as o3d
#from open3d.web_visualizer import draw
from scipy.spatial.transform import Rotation as Rot
import numpy as np
import os
from bop_toolkit_lib import config
from bop_toolkit_lib import dataset_params
from bop_toolkit_lib import inout
from bop_toolkit_lib import misc
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
dataset = "fidget_dataset"
mesh_directory = dataset+'/model_meshes'
mesh_list = {}
mesh_path_list =[]

for filename in os.listdir(mesh_directory):
    f = os.path.join(mesh_directory, filename)
    mesh_path_list.append(f)
    mesh = o3d.io.read_triangle_mesh(f)
    mesh_list[str(os.path.basename(f))]= mesh
    # checking if it is a file
    if os.path.isfile(f):
        print(f)

print(mesh_list)

assembling_order = ['bottom_casing.ply','left_gear.ply','right_gear.ply','top_casing.ply']
#mesh = o3d.io.read_triangle_mesh("top_casing.ply")

fidget_dataset/model_meshes/left_gear.ply
fidget_dataset/model_meshes/table1.ply
fidget_dataset/model_meshes/right_gear.ply
fidget_dataset/model_meshes/stage_2.ply
fidget_dataset/model_meshes/stage_1.ply
fidget_dataset/model_meshes/stage_3.ply
fidget_dataset/model_meshes/stage_4.ply
fidget_dataset/model_meshes/top_casing.ply
fidget_dataset/model_meshes/bottom_casing.ply
{'left_gear.ply': TriangleMesh with 9082 points and 18164 triangles., 'table1.ply': TriangleMesh with 19496 points and 39204 triangles., 'right_gear.ply': TriangleMesh with 12962 points and 25924 triangles., 'stage_2.ply': TriangleMesh with 11302 points and 22598 triangles., 'stage_1.ply': TriangleMesh with 2220 points and 4434 triangles., 'stage_3.ply': TriangleMesh with 24264 points and 48522 triangles., 'stage_4.ply': TriangleMesh with 25396 points and 50782 triangles., 'top_casing.ply': TriangleMesh with 1132 points and 2260 triangles., 'bottom_casing.ply': TriangleMesh with 2220 points and 4434 triangles.}


In [4]:
# Create the assembly
base_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.01, origin=[0, 0, 0])
transformations = {}

def adjust_mesh(name,color,R=[0,0,0],t=[0,0,0]):
    mesh = mesh_list[name]
    T_or = Rot.from_euler('zyx',R).as_matrix()
    T_pos = np.array(t).reshape(3,1)
    T= np.concatenate((T_or, T_pos), axis = 1) 
    T= np.vstack((T, [0,0,0,1] ))
    transformations[name] = T
    mesh.transform(T)
    mesh.paint_uniform_color(color)
    
    return mesh

bottom_casing = adjust_mesh(name='bottom_casing.ply', color=[0.561, 0.522, 0.255])
left_gear  = adjust_mesh(name='left_gear.ply', color=[0.561, 0.522, 0.255],t=[0.0115,0,0.005])
right_gear = adjust_mesh(name='right_gear.ply',color=[0.561, 0.522, 0.255],t=[-0.0115,0,0.005])
top_casing = adjust_mesh(name='top_casing.ply',color=[0.561, 0.522, 0.255],R =[0.0, 0.0, 3.14],t=[0.00, 0,0.012])


o3d.visualization.draw_geometries([bottom_casing, left_gear],mesh_show_wireframe=True)
#o3d.visualization.draw_geometries([bottom_casing, top_casing,left_gear, right_gear,base_frame],mesh_show_wireframe=True)


In [6]:
# generate intermediate assembly states and save them as mesh files
cm_1 = bottom_casing + left_gear
#o3d.io.write_triangle_mesh(mesh_directory+"/stage_2.ply", cm_1)

cm_2 = cm_1 + right_gear
#o3d.io.write_triangle_mesh(mesh_directory+"/stage_3.ply", cm_2)

cm_3 = cm_2 + top_casing
#o3d.io.write_triangle_mesh(mesh_directory+"/stage_4.ply", cm_3)

o3d.visualization.draw_geometries([bottom_casing], mesh_show_wireframe=True)

In [ ]:
# generate assembly_poses_gt.json
import json
with open("gt_assembly_poses.json", "w") as outfile:
    assembly_poses ={}
    for i, name in enumerate(assembling_order):
        obj_data = {
        "obj_id" : i,
        "obj_name": name,    
        "R_wrt_bottom_casing" : np.ravel(np.reshape(transformations[name][0:3,0:3],(1,9))).tolist(),
        "t_wrt_bottom_casing" : (transformations[name][0:3,3]*1000).tolist()
        }
        assembly_poses[i]= obj_data
    json.dump(assembly_poses, outfile)
    
#print(assembly_poses)

In [ ]:
models_info = {}
for obj_id, name in enumerate(assembling_order):
    
    misc.log('Processing model of object {}...'.format(name))
    
    model = inout.load_ply(mesh_directory+'/'+name)
    ref_pt = model['pts'].min(axis=0).flatten()
    size =(model['pts'].max(axis=0) - ref_pt)

    # Calculated diameter.
    diameter = misc.calc_pts_diameter(model['pts'])

    models_info[obj_id] = {
        'min_x': ref_pt[0], 'min_y': ref_pt[1], 'min_z': ref_pt[2],
        'size_x': size[0], 'size_y': size[1], 'size_z': size[2],
        'diameter': diameter
    }


# Save the calculated info about the object models.
inout.save_json(dataset+'/models_info.json', models_info)

In [ ]:
print(models_info)

In [ ]:
trans = misc.get_symmetry_transformations(models_info[0], max(15, models_info[0]['diameter']))

In [ ]:
print(trans)